In [1]:
!ls -lh /mnt/data2/PadChest/

total 306G
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 0.zip
-rw-r--r-- 1 pamessina pamessina 238K Oct 22 18:27 0.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 10.zip
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 10.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 11.zip
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 11.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 12.zip
-rw-r--r-- 1 pamessina pamessina 281K Oct 22 18:43 12.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:20 13.zip
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:18 13.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:16 14.zip
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:14 14.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:12 15.zip
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:12 15.zip.unzip-l.txt
-rw-r--r-- 1 pamessina pamessina    0 Oct 20 09:12 16.zip
-rw-r--r-- 1 pamessina pamessina 285K

In [22]:
import hashlib
import os
import subprocess
from termcolor import colored
from tqdm import tqdm

In [3]:
def load_sha1sums():
    file2sha1sum = dict()
    with open('/mnt/data2/PadChest/sha1sums.txt', 'r') as f:
        for line in f.readlines():
            sha1sum, filename = line.split()
            file2sha1sum[filename] = sha1sum
    return file2sha1sum

In [4]:
def compute_sha1(filepath):
    # BUF_SIZE is totally arbitrary, change for your app!
    BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

    sha1 = hashlib.sha1()

    with open(filepath, 'rb') as f:
        while True:
            data = f.read(BUF_SIZE)
            if not data:
                break
            sha1.update(data)
            
    return sha1.hexdigest()

In [5]:
def start(executable_file):
    return subprocess.Popen(
        executable_file,
        shell=True,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)

def read(process):
    return process.stdout.readline().decode("utf-8").strip()

def write(process, message):
    process.stdin.write(f"{message.strip()}\n".encode("utf-8"))
    process.stdin.flush()

def terminate(process):
    process.stdin.close()
    process.terminate()
    process.wait(timeout=0.2)

In [6]:
expected_sha1sums = load_sha1sums()

In [23]:
expected_sha1sums

{'0.zip': '249815521b04435271eb1ea473f64a8c48655392',
 '10.zip': '209283a99e481208d02c57225574fb6ac824d697',
 '11.zip': 'd18c752350b0f1443d563ee14b5862a6f991dc73',
 '12.zip': '7be008ac1296d65fba1e4f73ac9341a54eddf021',
 '13.zip': '68b921ad41c963677ae7080f5a885118205279a0',
 '14.zip': '2301e7f0b02eac86646c4b464ea416b4e320976b',
 '15.zip': 'b496dc8abc8f1f38e62e92bbf34373fff30330eb',
 '16.zip': '5109b5f485886c49ac508df95c496fe427e009e9',
 '17.zip': '3834dbaec03da167f24a4caaeddd2ccf829bf5b9',
 '18.zip': '5a56a3a33d17fc84d3306aabc7bf667746c5b5ab',
 '19.zip': '8e536a2c1ede845d2963291c31b6d9ba445dcbbe',
 '1.zip': 'e31e56df1a5f7314d6e44a989fe9a42155320a44',
 '20.zip': '362a7cbd18934157138bf0f18fc771d74320b4e4',
 '21.zip': 'a5181ba195f50b5d32de7e4bbbb9ceef0bda771d',
 '22.zip': '15e74429e5c8c4364e481aa0b3991b2b211355fd',
 '23.zip': '26db77ac6e8937a34949a877ee29da4a2f566e96',
 '24.zip': '363ef142020593baf7e7cdab570f1c9798ab9028',
 '25.zip': '5ac2987165df07cfea95d8bd6522ebbd3074845e',
 '26.zip': '

In [24]:
wrong_or_missing_filenames = []

In [25]:
existing_files = os.listdir('/mnt/data2/PadChest')

for filename in expected_sha1sums.keys():    
    if filename not in existing_files:        
        print('------------------------')
        print(colored(f'Missing file {filename}', 'blue'))
        wrong_or_missing_filenames.append(filename)

for filename in tqdm(existing_files):
    if filename in expected_sha1sums:
        
        filepath = os.path.join('/mnt/data2/PadChest', filename)
        print('------------------------')
        print(filepath)    
        exp_sha1 = expected_sha1sums[filename]
        actual_sha1 = compute_sha1(filepath)

        print(f'expected sha1:\t{exp_sha1}')
        print(f'actual sha1:\t{actual_sha1}')
        if actual_sha1 == exp_sha1:
            print(colored('OK!!!', 'green'))
        else:
            print(colored('Wrong hash :(', 'red'))
            print(f'Removing broken file {filepath}')
            os.remove(filepath)
            print(f'Done')
            wrong_or_missing_filenames.append(filename)

------------------------
Missing file 50.zip
------------------------
Missing file 50.zip.unzip-l.txt


  0%|                                                                                                     | 0/109 [00:00<?, ?it/s]

------------------------
/mnt/data2/PadChest/46.zip.unzip-l.txt


  1%|▊                                                                                            | 1/109 [00:00<01:10,  1.54it/s]

expected sha1:	57694ddeb4c620088c6fa66088477d665f16d179
actual sha1:	57694ddeb4c620088c6fa66088477d665f16d179
OK!!!
------------------------
/mnt/data2/PadChest/43.zip.unzip-l.txt
expected sha1:	9bce519e49de55e0ffc8c345dae1c6b68b76d40e
actual sha1:	9bce519e49de55e0ffc8c345dae1c6b68b76d40e
OK!!!
------------------------
/mnt/data2/PadChest/7.zip.unzip-l.txt
expected sha1:	c1fe8aa4f9662565dbc5ab6ac118cede1b32d8d5
actual sha1:	c1fe8aa4f9662565dbc5ab6ac118cede1b32d8d5
OK!!!
------------------------
/mnt/data2/PadChest/31.zip.unzip-l.txt
expected sha1:	5864b2fe370f2754c37f26c732ce344c82d36baa
actual sha1:	5864b2fe370f2754c37f26c732ce344c82d36baa
OK!!!
------------------------
/mnt/data2/PadChest/0.zip.unzip-l.txt
expected sha1:	ef2e9143eae375095ccba9951bf1154d8f8db380
actual sha1:	ef2e9143eae375095ccba9951bf1154d8f8db380
OK!!!
------------------------
/mnt/data2/PadChest/44.zip


...

expected sha1:	0550191bcaa56657f6c211f8dc147eb4a65c0b46
actual sha1:	0550191bcaa56657f6c211f8dc147eb4a65c0b46
OK!!!
------------------------
/mnt/data2/PadChest/1.zip


 99%|████████████████████████████████████████████████████████████████████████████████████████▏| 108/109 [1:28:32<01:10, 70.17s/it]

expected sha1:	e31e56df1a5f7314d6e44a989fe9a42155320a44
actual sha1:	e31e56df1a5f7314d6e44a989fe9a42155320a44
OK!!!
------------------------
/mnt/data2/PadChest/16.zip


100%|█████████████████████████████████████████████████████████████████████████████████████████| 109/109 [1:30:30<00:00, 49.83s/it]

expected sha1:	5109b5f485886c49ac508df95c496fe427e009e9
actual sha1:	5109b5f485886c49ac508df95c496fe427e009e9
OK!!!


In [26]:
wrong_or_missing_filenames

['50.zip', '50.zip.unzip-l.txt', '12.zip', '20.zip', '42.zip']

In [27]:
command_string = f'mget {" ".join(wrong_or_missing_filenames)}'
command_string

'mget 50.zip 50.zip.unzip-l.txt 12.zip 20.zip 42.zip'

In [30]:
wrong_or_missing_filenames_2 = []

for filename in tqdm(wrong_or_missing_filenames):
    if filename in expected_sha1sums:
        
        filepath = os.path.join('/mnt/data2/PadChest', filename)
        print('------------------------')
        print(filepath)
        try:
            exp_sha1 = expected_sha1sums[filename]
            actual_sha1 = compute_sha1(filepath)

            print(f'expected sha1:\t{exp_sha1}')
            print(f'actual sha1:\t{actual_sha1}')
            if actual_sha1 == exp_sha1:
                print(colored('OK!!!', 'green'))
            else:
                print(colored('Wrong hash :(', 'red'))
                print(f'Removing broken file {filepath}')
                os.remove(filepath)
                print(f'Done')
                wrong_or_missing_filenames_2.append(filename)
        except FileNotFoundError:
            print(colored(f'Missing file {filename}', 'blue'))
            wrong_or_missing_filenames_2.append(filename)            

  0%|                                                                                                       | 0/5 [00:00<?, ?it/s]

------------------------
/mnt/data2/PadChest/50.zip
expected sha1:	5d57691b4bd88a26818869a3da170eabaef8063c
actual sha1:	5d57691b4bd88a26818869a3da170eabaef8063c
OK!!!
------------------------
/mnt/data2/PadChest/50.zip.unzip-l.txt
expected sha1:	1c642224add0b74a7d9944dad29c1c9d41d79bdd
actual sha1:	1c642224add0b74a7d9944dad29c1c9d41d79bdd
OK!!!
------------------------
/mnt/data2/PadChest/12.zip


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.11s/it]

expected sha1:	7be008ac1296d65fba1e4f73ac9341a54eddf021
actual sha1:	7be008ac1296d65fba1e4f73ac9341a54eddf021
OK!!!
------------------------
/mnt/data2/PadChest/20.zip
Missing file 20.zip
------------------------
/mnt/data2/PadChest/42.zip
Missing file 42.zip


In [31]:
wrong_or_missing_filenames_2

['20.zip', '42.zip']

In [32]:
command_string = f'mget {" ".join(wrong_or_missing_filenames_2)}'
command_string

'mget 20.zip 42.zip'

In [28]:
# if len(wrong_or_missing_filenames) > 0:
#     cadaver_process = start("cd /mnt/data2/PadChest && cadaver https://b2drop.bsc.es/public.php/webdav/")
#     write(cadaver_process, 'BIMCV-PadChest-FULL') # username
#     write(cadaver_process, '') # password
#     write(cadaver_process, command_string)